In [91]:
import geopandas as gpd
import time
import os
import json
import requests
from osgeo import gdal
import pandas as pd
from requests.auth import HTTPBasicAuth
from dotenv import load_dotenv
import os
import requests
PLANET_API_KEY = "17de9ecc82734c1caf0ef0da1bf90d97"
import datetime


In [92]:
outpath = "data/imgs/"
if not os.path.exists(outpath):
    # Create a new directory because it does not exist 
    os.makedirs(outpath)


In [93]:
# Set filepath
fp = "shapes/boundary.shp"
# Read file using gpd.read_file()
data = gpd.read_file(fp)
#str(data.iloc[0]['geometry'])
# check the data
#gpd.GeoSeries([data.iloc[0]['geometry']])

In [94]:
#transforms to geoseries and then to json
a = json.loads(gpd.GeoSeries([data.iloc[0]['geometry']]).to_json())


In [95]:
#starts a session with planet api
session = requests.Session()
session.auth = (PLANET_API_KEY, '')


In [96]:
#creates the desired geometry and saves it into a file for cropping later
geojson_geometry = a['features'][0]['geometry']
import json
with open('subarea.geojson', 'w') as f:
    json.dump(geojson_geometry, f)


In [97]:
# get images that overlap with our AOI
geometry_filter = {
    "type": "GeometryFilter",
    "field_name": "geometry",
    "config": geojson_geometry
}

# get images acquired within a date range
date_range_filter = {
    "type": "DateRangeFilter",
    "field_name": "acquired",
    "config": {
        "gte": "2021-04-01T00:00:00.000Z",
        "lte": "2021-11-01T00:00:00.000Z"
    }
}

# only get images which have <50% cloud coverage
cloud_cover_filter = {
    "type": "RangeFilter",
    "field_name": "cloud_cover",
    "config": {
        "lte": 0.1
    }
}

# combine our geo, date, cloud filters
combined_filter = {
    "type": "AndFilter",
    "config": [geometry_filter, date_range_filter, cloud_cover_filter]
}

item_type = "PSScene3Band"


In [98]:

# API request object
search_request = {
    "name": "very_large_search",
    #"interval": "week",
    "item_types": [item_type],
    "filter": combined_filter
}

# fire off the POST request
search_result = \
    requests.post(
        'https://api.planet.com/data/v1/searches/',
        auth=HTTPBasicAuth(PLANET_API_KEY, ''),
        json=search_request)

#print(json.dumps(search_result.json(), indent=1))
search_result

<Response [200]>

In [99]:
from dateutil.parser import parse


In [100]:
dates_taken = {}

In [101]:
# What we want to do with each page of search results
# in this case, just print out each id
def handle_page(page):
    for image in page["features"]:
        id0 = image['id'] 
        datet = parse(image['properties']['acquired'])
        date_hash = f"{datet.day}_{datet.month}_{datet.year}"
        if date_hash in dates_taken:
            print(f"{date_hash} already done")
            continue
        dates_taken[date_hash] = 1
        output_file =  outpath + str(id0) + '_subarea.tif'
        output_file_xml =  outpath + str(id0) + '_subarea.xml'
        if os.path.exists(output_file):
            print(f"exists {output_file}")
            #continue
        image['filename'] = output_file
        image['analytics_filename'] = output_file_xml

        # For demo purposes, just grab the first image ID
        id0_url = 'https://api.planet.com/data/v1/item-types/{}/items/{}/assets'.format(
            item_type, id0)

        # Returns JSON metadata for assets in this ID. Learn more: planet.com/docs/reference/data-api/items-assets/#asset
        result = \
            requests.get(
                id0_url,
                auth=HTTPBasicAuth(PLANET_API_KEY, '')
            )

        # List of asset types available for this particular satellite image
        #print(result.json().keys())
        #print(result.json()['visual']['status'])

        # Parse out useful links
        links = result.json()[u"visual"]["_links"]
        self_link = links["_self"]
        activation_link = links["activate"]

        # Request activation of the 'visual' asset:
        activate_result = \
            requests.get(
                activation_link,
                auth=HTTPBasicAuth(PLANET_API_KEY, '')
            )


        activation_status_result = \
            requests.get(
                self_link,
                auth=HTTPBasicAuth(PLANET_API_KEY, '')
            )

        #print(activation_status_result.json()["status"])
        while activation_status_result.json()["status"] != "active":
            time.sleep(60)
            activation_status_result = \
                requests.get(
                    self_link,
                    auth=HTTPBasicAuth(PLANET_API_KEY, '')
                )
            print(activation_status_result.json()["status"])
        # Image can be downloaded by making a GET with your Planet API key, from here:
        #print(activation_status_result.json())
        download_link = activation_status_result.json()["location"]
        #print(download_link)
        vsicurl_url = '/vsicurl/' + download_link
        # GDAL Warp crops the image by our AOI, and saves it
        gdal.Warp(output_file, vsicurl_url, dstSRS = 'EPSG:4326', cutlineDSName = 'subarea.geojson', cropToCutline = True)

        
        
        # band4
        # For demo purposes, just grab the first image ID
        id0_url = 'https://api.planet.com/data/v1/item-types/{}/items/{}/assets'.format(
            "PSScene4Band", id0)

        # Returns JSON metadata for assets in this ID. Learn more: planet.com/docs/reference/data-api/items-assets/#asset
        result = \
            requests.get(
                id0_url,
                auth=HTTPBasicAuth(PLANET_API_KEY, '')
            )

        # List of asset types available for this particular satellite image
        #print(result.json().keys())
        #print(result.json()['visual']['status'])
        #print(result.json())
        # Parse out useful links
        links = result.json()[u"analytic_xml"]["_links"]
        self_link = links["_self"]
        activation_link = links["activate"]

        # Request activation of the 'visual' asset:
        activate_result = \
            requests.get(
                activation_link,
                auth=HTTPBasicAuth(PLANET_API_KEY, '')
            )


        activation_status_result = \
            requests.get(
                self_link,
                auth=HTTPBasicAuth(PLANET_API_KEY, '')
            )

        #print(activation_status_result.json()["status"])
        while activation_status_result.json()["status"] != "active":
            activation_status_result = \
                requests.get(
                    self_link,
                    auth=HTTPBasicAuth(PLANET_API_KEY, '')
                )
            print(activation_status_result.json()["status"])
            time.sleep(60)
        # Image can be downloaded by making a GET with your Planet API key, from here:
        #print(activation_status_result.json())
        download_link = activation_status_result.json()["location"]
        #print(download_link)
        r = requests.get(download_link)
        image['analytics_size'] = len(r.content)
        open(output_file_xml, 'wb').write(r.content)


        out_results.append(image)


In [ ]:
out_results = []

# after you create a search, save the id. This is what is needed
# to execute the search.
search_result_id = search_result.json()["id"]


# How to Paginate:
# 1) Request a page of search results
# 2) do something with the page of results
# 3) if there is more data, recurse and call this method on the next page.
def fetch_page(search_url):
    page = session.get(search_url).json()
    handle_page(page)
    next_url = page["_links"].get("_next")
    if next_url:
        fetch_page(next_url)

first_page = \
    ("https://api.planet.com/data/v1/searches/{}" +
        "/results?_page_size={}").format(search_result_id, 25)

# kick off the pagination
fetch_page(first_page)

# extract image IDs only
#image_ids = [feature['id'] for feature in search_result.json()['features']]
#print(image_ids)
#print(len(image_ids))


26_10_2021 already done
20_10_2021 already done
20_10_2021 already done
activating
activating
activating
activating
activating
activating
activating
activating
activating
activating
activating
active


In [ ]:
#from osgeo import gdal
#results = search_result.json()
#images = results['features']

In [ ]:
# transform the results into a nice list of dicts
p_results = []
for out in out_results:
    p = {
        'id': out['id'],
        'filename': out['filename'],
        'analytics_filename': out['analytics_filename'],
        'analytics_size': out['analytics_size'],
    }
    p = dict(list(p.items()) + list(out['properties'].items()))
    p['geometry']  = out['geometry']
    p_results.append(p)

In [ ]:
#saves results as a csv
import pandas as pd
df = pd.DataFrame(p_results)
df.to_csv('data/res.csv', index=None)

In [ ]:
out_results = []
for image in images[:10]:
    id0 = image['id'] 
    output_file = 'data/' + str(id0) + '_subarea.tif'
    image['filename'] = output_file
    out_results.append(image)
    # For demo purposes, just grab the first image ID
    id0_url = 'https://api.planet.com/data/v1/item-types/{}/items/{}/assets'.format(
        item_type, id0)

    # Returns JSON metadata for assets in this ID. Learn more: planet.com/docs/reference/data-api/items-assets/#asset
    result = \
        requests.get(
            id0_url,
            auth=HTTPBasicAuth(PLANET_API_KEY, '')
        )

    # List of asset types available for this particular satellite image
    #print(result.json().keys())
    #print(result.json()['visual']['status'])

    # Parse out useful links
    links = result.json()[u"visual"]["_links"]
    self_link = links["_self"]
    activation_link = links["activate"]

    # Request activation of the 'visual' asset:
    activate_result = \
        requests.get(
            activation_link,
            auth=HTTPBasicAuth(PLANET_API_KEY, '')
        )


    activation_status_result = \
        requests.get(
            self_link,
            auth=HTTPBasicAuth(PLANET_API_KEY, '')
        )

    #print(activation_status_result.json()["status"])
    while activation_status_result.json()["status"] != "active":
        time.sleep(20)
        activation_status_result = \
            requests.get(
                self_link,
                auth=HTTPBasicAuth(PLANET_API_KEY, '')
            )
        print(activation_status_result.json()["status"])
    # Image can be downloaded by making a GET with your Planet API key, from here:
    download_link = activation_status_result.json()["location"]
    #print(download_link)
    vsicurl_url = '/vsicurl/' + download_link
    # GDAL Warp crops the image by our AOI, and saves it
    gdal.Warp(output_file, vsicurl_url, dstSRS = 'EPSG:4326', cutlineDSName = 'subarea.geojson', cropToCutline = True)

